In [1]:
from SimulationSrc import *
import openai
from dotenv import load_dotenv

load_dotenv('.env')
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
results = []
# 共告知day天的数据
for day in range(1, 6):
    # 重复模拟20次
    for idx in range(1, 21):
        sl = NewsboySimulator()
        # data = sl.simulate_constOrder(day)
        data = sl.simulate(day)

        print(f'{day} day {idx} times: {data}')
        DemandLambda = sl.get_DemandLambda()
        OrderLambda = sl.get_OrderLambda()
        # 用三种方式推断最佳订货量
        poissonInfer = PoissonInfer(data)
        poissonresult = poissonInfer.infer()
        
        empricalInfer = EmpiricalInfer(data)
        empricalresult = empricalInfer.infer()

        chatGPTInfer = ChatGPTInfer(data)
        chatGPTresponse = chatGPTInfer.infer_cot()

        # 最后一行是结果，是单个数字
        chatGPTresult = chatGPTresponse[-1]

        # 同时保存 data
        results.append([day, idx, data, DemandLambda, OrderLambda, poissonresult, empricalresult ,chatGPTresult, chatGPTresponse])
        print('finish', day, idx)
# 保存结果
import csv
with open('./results/result_0228_var_cot.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['day', 'idx', 'SampleData', 'DemandLambda', 'OrderLambda' ,'poisson', 'empirical', 'chatGPT', 'GPTresponse'])
    for row in results:
        writer.writerow(row)



1 day 1 times: [(12, 3)]
finish 1 1
1 day 2 times: [(4, 2)]
finish 1 2
1 day 3 times: [(2, 2)]
0.12088084114111315
4.672569093473346
0.06552089854710184
5.549281904709877
0.04469926100886568
6.08733778420068
0.033807163197515935
6.474868129410964
0.02712523240950926
6.777102181569143
0.022615728196888868
7.024449359009979
0.019371569996781735
7.2335512356424285
0.01692811621145355
7.414500424039431
0.015022950021615949
7.573875264128468
0.013496771743049868
7.716199490682374
0.012247340269641705
7.844715701524709
0.011206060045953843
7.961826427147703
0.010325213112888011
8.069360605043453
0.00957059018291278
8.168742273943488
0.008917034069643986
8.261101575456687
0.008345636425559442
8.347350208224746
0.00784191435775405
8.428234162560953
0.007394589600216949
8.504371471553988
0.006994750233459816
8.576279808078139
0.006635262150475963
8.644397034698724
0.006310347637645107
8.709096758812432
0.006015278256492392
8.770700280768157
0.005746147452517037
8.829485893095299
0.0054996997668

In [6]:
with open('./results/result_0228_var_cot_org.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['day', 'idx', 'SampleData', 'DemandLambda', 'OrderLambda' ,'poisson', 'empirical', 'chatGPT', 'GPTresponse'])
    for row in results:
        writer.writerow(row)


In [11]:
import pandas as pd
from SimulationSrc import *
df = pd.read_csv('./results/result_0228_var_cot.csv',encoding='utf-8-sig',)
# 把ChatGPT列中为'None'的行删除
# df = df[df['chatGPT'] != 'None']
df = df[df['poisson'] != 'nan']
df = df[df['chatGPT'] != 'nan']
# dropna
df = df.dropna()
# df = df[df['poisson'] !='NAN']
# 遍历每一行
# day,idx,DemandLambda,OrderLambda,poisson,emprical,chatGPT
for index, row in df.iterrows():
    DemandLambda = float(row['DemandLambda'])

    poisson_order = int(row['poisson'])
    emprical_order = int(row['empirical'])
    chatGPT_order = int(row['chatGPT'])

    poisson_cost = get_average_cost(poisson_order, DemandLambda)
    emprical_cost = get_average_cost(emprical_order, DemandLambda)
    chatGPT_cost = get_average_cost(chatGPT_order, DemandLambda)


    # 把三种方法的cost和ChatGPT是否beat了poisson和emprical写入csv
    df.at[index, 'poisson_cost'] = poisson_cost
    df.at[index, 'empirical_cost'] = emprical_cost
    df.at[index, 'ChatGPT_cost'] = chatGPT_cost

    # # 判断ChatGPT是否beat了poisson和emprical
    # if chatGPT_cost < poisson_cost:
    #     df.at[index, 'ChatGPT_beat_poisson'] = 1
    # if chatGPT_cost == poisson_cost:
    #     df.at[index, 'ChatGPT_equal_poisson'] = 1

    # if chatGPT_cost < emprical_cost:
    #     df.at[index, 'ChatGPT_beat_empirical'] = 1
    # if chatGPT_cost == emprical_cost:
    #     df.at[index, 'ChatGPT_equal_empirical'] = 1
# 在最后一行补上三种方法的平均cost
poisson_average_cost = df['poisson_cost'].mean()
emprical_average_cost = df['empirical_cost'].mean()
chatGPT_average_cost = df['ChatGPT_cost'].mean()

# 新开一行，记录三种方法的平均cost
output_name = './results/result_0228_var_cot_withcost.csv'
df.to_csv(output_name, index=False)
with open(output_name, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([None, None, None, None, None, None, None,None, 'average_cost', poisson_average_cost, emprical_average_cost, chatGPT_average_cost])